For the input of pyclone (https://github.com/Roth-Lab/pyclone) prepar a table that includes all the SNVs from the 2 timepoints the samples were collected and for each SNV set the normal copy number (normal_cn) to 2 (unless it is a variant on the Y chromosome then normal_cn is set to 1). For each variant check if it is located in regions of CNVs (using the CNV_FACETS output - https://github.com/dariober/cnv_facets#variants). So if the variant was where a CNV is located, the copy number of the minor allele in the malignant cells (minor_cn) and the copy number of the major allele in the malignant cells (major_cn) are set to the values "LCN_EM" and "TCN_EM - LCN_EM" from the CNV_facets output.

This script is run in a python 3.5 environment.

In [4]:
import vcf
import pandas as pd
import numpy as np

In [2]:
def pyclone_prep(T_sample_snvs, T_sample_cnvs, path_to_cnvs, genotype):
    
    d = {'mutation_id' : [],
        'ref_counts' : [],
        'var_counts' : [],
        'normal_cn' : [],
        'minor_cn' : [], 
        'major_cn' : []
        }
    

    for record in T_sample_snvs:
        #print(record)
        chrom = record.CHROM
        pos = record.POS
        ref = record.REF
        alt = record.ALT[0]  ##because it has the form of a list
        rsid = record.ID
        AD_ref = record.genotype(genotype).data.AD[0]
        AD_alt = record.genotype(genotype).data.AD[1]
    
    
        if rsid == None:
            mutation_ID = str(chrom) + ":" + str(pos) + ":" + str(ref) + ":" + str(alt)
        else:
            mutation_ID = str(chrom) + ":" + str(pos) + ":" + str(ref) + ":" + str(alt) + ":" + str(rsid)
          
            
        if chrom == "chrY":
            normal_cn = 1
        else:
            normal_cn = 2

    
    
        T_sample_cnvs = vcf.Reader(open(path_to_cnvs, 'r'))
        for record_ in T_sample_cnvs:
            chrom_ = record_.CHROM
            pos_ = record_.POS
            end_ = record_.INFO['END']
        
            tcn_ = record_.INFO['TCN_EM']
            minor_cn_ = record_.INFO['LCN_EM']
            if minor_cn_ == None:
                minor_cn_ = 0
            major_cn_ = tcn_ - minor_cn_
   
            if chrom == chrom_:
                if pos >= pos_ and pos <= end_ :
                    minor_cn = minor_cn_
                    major_cn = major_cn_
                else:
                    minor_cn = 1
                    major_cn = 1
                

        d['mutation_id'].append(mutation_ID)
        d['ref_counts'].append(AD_ref)
        d['var_counts'].append(AD_alt)
        d['normal_cn'].append(normal_cn)
        d['minor_cn'].append(minor_cn)
        d['major_cn'].append(major_cn)
        
    
    df = pd.DataFrame(d)
    
    return df

    

When the one of the input files is a maf file, use the following function.
This will be useful when it is needed to see the gene names for each SNV after pyclone has run.

This function is for the maf files after integration of the 2 variant callers: Mutect2 and Strelka. 

1. The count of ref and alt alleles are stored in the columns: t_Mutect_Counts t_Strelka_Counts (comma separated). If empty use only the caller that has found the variant. If the variant has been called by both callers take the average and round it!

2. Entries in the vcf file called by cnv_facets with entries with: “LCN_EM=.” are handled replacing the value with 1. Probably focal changes. Check the output if it gieves sensible answers. Otherwise maybe filter out this CNV (?).


In [50]:
def pyclone_prep_maf2(T_sample_snvs, T_sample_cnvs, path_to_cnvs, genotype):
    
    d = {'mutation_id' : [],
        'ref_counts' : [],   
        'var_counts' : [],   
        'normal_cn' : [],
        'minor_cn' : [], 
        'major_cn' : []
        }
    
     
    # loop through the rows using iterrows()
    for index, row in T_sample_snvs.iterrows():
        gene = row['Hugo_Symbol']
        var_class = row['Variant_Classification']
        chrom = row['Chromosome']
        pos = int(row['Start_Position'])
        ref = row['Reference_Allele'] 
        alt = row['Tumor_Seq_Allele2']
        rsid = row['dbSNP_RS']
        mutect_counts = row['t_Mutect_Counts']  
        strelka_counts = row['t_Strelka_Counts']  

    
        mutation_ID = str(gene) + "|" + str(var_class) + ":" + str(chrom) + ":" + str(pos) + ":" + str(ref) + ":" + str(alt) + ":" + str(rsid)
            
        if chrom == "chrY":
            normal_cn = 1
        else:
            normal_cn = 2

    
    
        T_sample_cnvs = vcf.Reader(open(path_to_cnvs, 'r'))
        for record_ in T_sample_cnvs:
            chrom_ = record_.CHROM
            pos_ = record_.POS
            end_ = record_.INFO['END']
            
            svtype_ = record_.INFO['SVTYPE']
            
            tcn_ = record_.INFO['TCN_EM']
            
            #print(chrom_, pos_, end_, tcn_, svtype_)
            
            minor_cn_ = record_.INFO['LCN_EM']
            # handling entries with:  with “LCN_EM=.”
            if minor_cn_ == None: 
                minor_cn_ = 1
            major_cn_ = tcn_ - minor_cn_
            
            if chrom == chrom_ and pos >= pos_ and pos <= end_ :
                #print("IN RANGE")
                minor_cn = minor_cn_
                major_cn = major_cn_
                
                break
            
            
            else:
                minor_cn = 1
                major_cn = 1
       
        
        
        if mutect_counts == ".,.": ##or NA
            counts = [strelka_counts]
        elif strelka_counts == ".,.":
            counts = [mutect_counts]
        else:
            counts = [strelka_counts, mutect_counts]
     

        if len(counts) == 1:
            count_values = counts[0].strip().split(",")
            AD_ref = count_values[0]
            AD_alt = count_values[1]
        else:
            count_values_strelka = counts[0].strip().split(",")
            AD_ref_strelka = count_values_strelka[0]
            AD_alt_strelka = count_values_strelka[1]
            count_values_mutect = counts[1].strip().split(",")
            AD_ref_mutect = count_values_mutect[0]
            AD_alt_mutect = count_values_mutect[1]

            AD_ref = np.round(np.mean([int(AD_ref_strelka),int(AD_ref_mutect)])).astype(int)
            AD_alt = np.round(np.mean([int(AD_alt_strelka),int(AD_alt_mutect)])).astype(int)
                       
        
                

        d['mutation_id'].append(mutation_ID)
        d['ref_counts'].append(AD_ref)
        d['var_counts'].append(AD_alt)
        d['normal_cn'].append(normal_cn)
        d['minor_cn'].append(minor_cn)
        d['major_cn'].append(major_cn)
        
    
    df = pd.DataFrame(d)
    
    return df

    

Example using a MAF file to extract the SNVs

In [51]:
T_sampleID_snvs = pd.read_csv('/home/rania/Documents/Tina/WES/pyclone/T_650/01_pyclone_prep/T_650_1.maf',
                              delimiter='\t',
                              skiprows=None,
                              header=0
                             )

T_sampleID_cnvs = vcf.Reader(open('/home/rania/Documents/Tina/WES/pyclone/T_650/01_pyclone_prep/T_650_1.vcf', 'r'))


df = pyclone_prep_maf2(T_sample_snvs = T_sampleID_snvs, T_sample_cnvs = T_sampleID_cnvs, path_to_cnvs = '/home/rania/Documents/Tina/WES/pyclone/T_650/01_pyclone_prep/T_650_1.vcf', genotype = 'T_650_1')

df.to_csv('/home/rania/Documents/Tina/WES/pyclone/T_650/02_pyclone_input/T_650_1.tsv', sep="\t", index = False)

In [52]:
T_sampleID_snvs = pd.read_csv('/home/rania/Documents/Tina/WES/pyclone/T_650/01_pyclone_prep/T_650_2.maf',
                              delimiter='\t',
                              skiprows=None,
                              header=0
                             )

T_sampleID_cnvs = vcf.Reader(open('/home/rania/Documents/Tina/WES/pyclone/T_650/01_pyclone_prep/T_650_2.vcf', 'r'))


df = pyclone_prep_maf2(T_sample_snvs = T_sampleID_snvs, T_sample_cnvs = T_sampleID_cnvs, path_to_cnvs = '/home/rania/Documents/Tina/WES/pyclone/T_650/01_pyclone_prep/T_650_2.vcf', genotype = 'T_650_1')

df.to_csv('/home/rania/Documents/Tina/WES/pyclone/T_650/02_pyclone_input/T_650_2.tsv', sep="\t", index = False)

In [53]:
T_sampleID_snvs = pd.read_csv('/home/rania/Documents/Tina/WES/pyclone/T_650/01_pyclone_prep/T_650_3.maf',
                              delimiter='\t',
                              skiprows=None,
                              header=0
                             )

T_sampleID_cnvs = vcf.Reader(open('/home/rania/Documents/Tina/WES/pyclone/T_650/01_pyclone_prep/T_650_3.vcf', 'r'))


df = pyclone_prep_maf2(T_sample_snvs = T_sampleID_snvs, T_sample_cnvs = T_sampleID_cnvs, path_to_cnvs = '/home/rania/Documents/Tina/WES/pyclone/T_650/01_pyclone_prep/T_650_3.vcf', genotype = 'T_650_1')

df.to_csv('/home/rania/Documents/Tina/WES/pyclone/T_650/02_pyclone_input/T_650_3.tsv', sep="\t", index = False)

In [54]:
T_sampleID_snvs = pd.read_csv('/home/rania/Documents/Tina/WES/pyclone/T_608/01_pyclone_prep/T1_608.maf',
                              delimiter='\t',
                              skiprows=None,
                              header=0
                             )

T_sampleID_cnvs = vcf.Reader(open('/home/rania/Documents/Tina/WES/pyclone/T_608/01_pyclone_prep/T1_608.vcf', 'r'))


df = pyclone_prep_maf2(T_sample_snvs = T_sampleID_snvs, T_sample_cnvs = T_sampleID_cnvs, path_to_cnvs = '/home/rania/Documents/Tina/WES/pyclone/T_608/01_pyclone_prep/T1_608.vcf', genotype = 'T1_608')

df.to_csv('/home/rania/Documents/Tina/WES/pyclone/T_608/02_pyclone_input/T1_608.tsv', sep="\t", index = False)

In [55]:
T_sampleID_snvs = pd.read_csv('/home/rania/Documents/Tina/WES/pyclone/T_608/01_pyclone_prep/T2_608.maf',
                              delimiter='\t',
                              skiprows=None,
                              header=0
                             )

T_sampleID_cnvs = vcf.Reader(open('/home/rania/Documents/Tina/WES/pyclone/T_608/01_pyclone_prep/T2_608.vcf', 'r'))


df = pyclone_prep_maf2(T_sample_snvs = T_sampleID_snvs, T_sample_cnvs = T_sampleID_cnvs, path_to_cnvs = '/home/rania/Documents/Tina/WES/pyclone/T_608/01_pyclone_prep/T2_608.vcf', genotype = 'T2_608')

df.to_csv('/home/rania/Documents/Tina/WES/pyclone/T_608/02_pyclone_input/T2_608.tsv', sep="\t", index = False)

In [56]:
T_sampleID_snvs = pd.read_csv('/home/rania/Documents/Tina/WES/pyclone/T_588/01_pyclone_prep/T_588_1.maf',
                              delimiter='\t',
                              skiprows=None,
                              header=0
                             )

T_sampleID_cnvs = vcf.Reader(open('/home/rania/Documents/Tina/WES/pyclone/T_588/01_pyclone_prep/T_588_1.vcf', 'r'))


df = pyclone_prep_maf2(T_sample_snvs = T_sampleID_snvs, T_sample_cnvs = T_sampleID_cnvs, path_to_cnvs = '/home/rania/Documents/Tina/WES/pyclone/T_588/01_pyclone_prep/T_588_1.vcf', genotype = 'T_588_1')

df.to_csv('/home/rania/Documents/Tina/WES/pyclone/T_588/02_pyclone_input/T_588_1.tsv', sep="\t", index = False)

In [57]:
T_sampleID_snvs = pd.read_csv('/home/rania/Documents/Tina/WES/pyclone/T_588/01_pyclone_prep/T_588_2.maf',
                              delimiter='\t',
                              skiprows=None,
                              header=0
                             )

T_sampleID_cnvs = vcf.Reader(open('/home/rania/Documents/Tina/WES/pyclone/T_588/01_pyclone_prep/T_588_2.vcf', 'r'))


df = pyclone_prep_maf2(T_sample_snvs = T_sampleID_snvs, T_sample_cnvs = T_sampleID_cnvs, path_to_cnvs = '/home/rania/Documents/Tina/WES/pyclone/T_588/01_pyclone_prep/T_588_2.vcf', genotype = 'T_588_2')

df.to_csv('/home/rania/Documents/Tina/WES/pyclone/T_588/02_pyclone_input/T_588_2.tsv', sep="\t", index = False)

Example of using the above function with inputs vcf files (after SNV and CNV calling):

In [ ]:
T1_sampleID_snvs = vcf.Reader(open('path/to/T1_sampleID.strelka_muTect.somatic.snv.vcf', 'r'))

T1_sampleID_cnvs = vcf.Reader(open('path/to/T1_sampleID.vcf', 'r'))


df = pyclone_prep(T_sample_snvs = T1_sampleID_snvs, T_sample_cnvs = T1_sampleID_cnvs, path_to_cnvs = 'path/to/T1_sampleID.vcf', genotype = 'T1_sampleID')

df.to_csv('T1_sampleID.tsv', sep="\t", index = False)

In [ ]:
T2_sampleID_snvs = vcf.Reader(open('path/to/T2_sampleID.strelka_muTect.somatic.snv.vcf', 'r'))

T2_sampleID_cnvs = vcf.Reader(open('path/to/T2_sampleID.vcf', 'r'))


df = pyclone_prep(T_sample_snvs = T2_sampleID_snvs, T_sample_cnvs = T2_sampleID_cnvs, path_to_cnvs = 'path/to/T1_sampleID.vcf', genotype = 'T2_sampleID')

df.to_csv('T2_sampleID.tsv', sep="\t", index = False)